In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import Sequence
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import math
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt

In [2]:
# Function to Extract features from the images
def image_feature(direc):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = []
    img_name = []
    for i in tqdm(direc):
        fname='./archive/train/train/'+i
        img=image.load_img(fname,target_size=(224,224))
        x = img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        feat=model.predict(x)
        feat=feat.flatten()
        features.append(feat)
        img_name.append(i)
    return features,img_name

In [3]:
img_path=os.listdir('./archive/train/train/') 
img_features,img_name=image_feature(img_path)

100%|██████████| 1400/1400 [02:00<00:00, 11.61it/s]


In [10]:
# Creating Clusters
k = 2
clusters = KMeans(k, random_state = 40)
clusters.fit(img_features)

KMeans(n_clusters=2, random_state=40)

In [11]:
image_cluster = pd.DataFrame(img_name,columns=['image'])
image_cluster["clusterid"] = clusters.labels_
image_cluster # 0 denotes cat and 1 denotes dog

,image,clusterid
0,cat.1.jpg,0
1,cat.10.jpg,0
2,cat.100.jpg,0
3,cat.101.jpg,0
4,cat.102.jpg,0
...,...,...
1395,dog.95.jpg,1
1396,dog.96.jpg,1
1397,dog.97.jpg,1
1398,dog.98.jpg,1


In [12]:
# Images will be seperated according to cluster they belong
for i in range(len(image_cluster)):
    if image_cluster['clusterid'][i]==0:
        np.save("./archive/process/cats/cats_array", image_cluster)
    else:
        np.save("./archive/process/dogs/dogs_array", image_cluster)

In [13]:
cat_array = np.load("./archive/process/cats/cats_array.npy",allow_pickle=True)
dogs_array = np.load("./archive/process/dogs/dogs_array.npy",allow_pickle=True)
cat_array.shape, dogs_array.shape

((1400, 2), (1400, 2))